# DGX Kernel Root Cause Analysis Acceleration & Predictive Maintenance using NLP

# Table of Contents 
* Introduction
* Dataset
* Reading in the datasets
* Initialize/Load BERT model
* Training - DGX Kernel logs dataset
* Evaluation
* Conclusion
* References

# Introduction

Like any other Linux based machine, DGX's generate a vast amount of logs. Analysts spend hours trying to identify the root causes of each failure. There could be infinitely many types of root causes of the failures. Some patterns might help to narrow it down; however, regular expressions can only help to identify previously known patterns. Moreover, this creates another manual task of maintaining a search script. 

In this notebook, we show how GPU's can accelerate the analysis of the enormous amount of logs using machine learning. Another benefit of analyzing in a probabilistic way is that we can pin down previously undetected root causes. To achieve this, we will fine-tune a pre-trained BERT* model with a classification layer using HuggingFace library.

Once the model is capable of identifying even the new root causes, it could also be deployed as a process running in the machines to predict failures before they happen.

*BERT stands for Bidirectional Encoder Representations from Transformers. The paper can be found [here.](https://arxiv.org/pdf/1810.04805.pdf)

## Dataset
* DGX Linux Kernel logs

The dataset comprises `kern.log` files from multiple DGX's. Each line inside has been labelled as either `0` for `ordinary` or `1` for `root cause` by a script that uses some known patterns. We will be especially interested in lines that are marked as ordinary in the test set but predicted as a root cause, as they may be new types of root causes of failures.

More information on Linux log types can be found [here.](https://help.ubuntu.com/community/LinuxLogFiles)

### Required Libraries

In [1]:
import cudf
from cudf.core.subword_tokenizer import SubwordTokenizer
from os import path;
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import torch
import sys

from morpheus.utils.seed import manual_seed
from sequence_classifier import SequenceClassifier

## Reading the files

In [2]:
random_seed=42

In [3]:
dflogs = pd.read_csv("./kernel_sampled_multiclass.tsv", delimiter='\t', header=None,names=["label","log"])

In [4]:
dflogs.head()

,label,log
0,0,kernel: [ 27.589832] pci_bus 0000:ba: resou...
1,0,kernel: [ 28.139585] pci 0000:00:19.2: Addi...
2,0,kernel: [90396.231320] docker0: port 1(veth6a...
3,0,kernel: [10559.914471] docker0: port 1(vethfc...
4,0,kernel: [1103999.101610] dl_rq[176]:


In [14]:
dflogs["label"].value_counts()

0    50000
2    24690
3     1278
1      917
Name: label, dtype: int64

Each row in the `log` column has a line from the `kern.log` file, and the label column has information on whether it is an ordinary log or a root cause.

Previously unseen error types are read into a new dataframe to be appended to the test set later.

In [ ]:
# dfnewerror=pd.read_csv("../../datasets/training-data/root-cause-unseen-errors.csv", header=None, names=['label', 'log'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(dflogs, dflogs.label, random_state=random_seed)

In [ ]:
# X_train.reset_index(drop=True,inplace=True)

In [ ]:
# y_train.reset_index(drop=True,inplace=True)

In [ ]:
# X_test=pd.concat([X_test,dfnewerror])

In [ ]:
# y_test=pd.concat([y_test,dfnewerror["label"]])

In [ ]:
# X_test.reset_index(drop=True,inplace=True)

In [ ]:
# X_train.to_json("Rootcause-training-data.jsonlines", orient='records',lines=True)

In [ ]:
# X_test.to_json("Rootcause-validation-data.jsonlines", orient='records',lines=True)

In [ ]:
# y_test.reset_index(drop=True,inplace=True)

In [ ]:
# dfnewerror=cudf.DataFrame.from_pandas(dfnewerror)

In [ ]:
dflogs = cudf.DataFrame.from_pandas(dflogs)

## Initialize/Load the BERT model
We will initialize the sequence classifier module with a pre-trained BERT model. The pre-trained model we use is located at https://huggingface.co/bert-base-uncased For more information on the model, please see the paper at https://arxiv.org/pdf/2005.01634.pdf


In [6]:
seq_classifier = SequenceClassifier("bert-base-uncased", hash_file="../../../morpheus/data/bert-base-uncased-hash.txt", num_labels=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Training 

Part of the dataset will be used for fine-tuning the model. The rest of the dataset will be used as the test set to evaluate if the model is useful. With default settings, 80% of the dataset will be the training set.

The number of epochs should be adjusted for each dataset.

In [7]:
manual_seed(random_seed)
seq_classifier.train_model(X_train["log"], y_train,batch_size=128, epochs=1, learning_rate=3.6e-4)

Epoch: 100%|██████████| 1/1 [06:43<00:00, 403.50s/it]

Train loss: 0.6093593457932615


In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")

seq_classifier.save_model(timestr)

## Evaluation of the model

`evaluate_model` returns the accuracy in the test set.

We will check if any of the error lines we appended are predicted as root-cause

In [9]:
seq_classifier.evaluate_model(X_test["log"], y_test)

0.6496936923309635

We get the predictions from the model.

In [15]:
test_preds = seq_classifier.predict(X_test["log"], batch_size=128)

/opt/conda/envs/morpheus/lib/python3.10/site-packages/cudf/core/indexed_frame.py:3424: FutureWarning: The append method is deprecated and will be removed in a future version. Use cudf.concat instead.
  warnings.warn(


In [11]:
# tests = test_preds.to_numpy()
# true_labels = X_test["label"]

In [17]:
tests = test_preds.to_numpy()
true_labels = X_test.label.to_numpy()

Calculate the F1 score since it's not a balanced dataset.

In [18]:
f1_score(true_labels, tests, average=None)

array([0.7876762, 0.       , 0.       , 0.       ])

Accuracy is higher than the F1 score. The distribution of the labels is not balanced hence accuracy might be less indicative of performance.

We can use a confusion matrix to check how many of each label are predicted as marked.

In [19]:
confusion_matrix(true_labels, tests)

array([[12489,     0,     0,     0],
       [  232,     0,     0,     0],
       [ 6183,     0,     0,     0],
       [  318,     0,     0,     0]])

## Export model to ONNX

In [ ]:
tokenizer = SubwordTokenizer("../../../morpheus/data/bert-base-uncased-hash.txt", do_lower_case=True)

In [ ]:
tokenizer_output = tokenizer(dflogs["log"][0:3],
                                max_length=128,
                                max_num_rows=3,
                                truncation=True,
                                add_special_tokens=True,
                                return_tensors="pt")

sample_model_input = (tokenizer_output["input_ids"].type(torch.long), tokenizer_output["attention_mask"].type(torch.long))

In [ ]:
torch.onnx.export(seq_classifier._model,              
                  sample_model_input,               
                  "model.onnx",                                      # where to save the model
                  export_params=True,                                # store the trained parameter weights inside the model file
                  opset_version=10,                                  # the ONNX version to export the model to
                  do_constant_folding=True,                          # whether to execute constant folding for optimization
                  input_names = ['input_ids','attention_mask'],      # the model's input names
                  output_names = ['output'],                         # the model's output names
                  dynamic_axes={'input_ids' : {0 : 'batch_size'},    # variable length axes
                                'attention_mask': {0: 'batch_size'}, 
                                'output' : {0 : 'batch_size'}})

# Conclusion

Although we didn't include hardware errors in the training set, the model still marks some of them as root cause lines which can be seen from the indices of the positive predictions above. The lines identified by this model may indicate a problem hours before the failure or outage. This approach can be implemented on the machines to warn the users well before the problems occur so corrective actions can be taken.

# References
* https://github.com/huggingface/transformers/tree/master/examples#
* BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding https://arxiv.org/pdf/1810.04805.pdf